# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

To see this example online, view the [JavaScript version][js-example].

[js-example]: https://earthengine-layers.com/examples/image

To run this notebook locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, make sure you have [Conda installed](https://www.anaconda.com/products/individual), then run:

```bash
conda create -n pydeck-ee -c conda-forge python jupyter jupyterlab notebook pydeck earthengine-api nodejs -y
conda activate pydeck-ee

# Install the pydeck-earthengine-layers package from pip
pip install pydeck-earthengine-layers

# If using Jupyter Notebook:
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck

# If using Jupyter Lab
jupyter labextension install @jupyter-widgets/jupyterlab-manager
DECKGL_SEMVER=`python -c "import pydeck; print(pydeck.frontend_semver.DECKGL_SEMVER)"`
jupyter labextension install @deck.gl/jupyter-widget@$DECKGL_SEMVER
```

then open Jupyter Notebook with `jupyter notebook` or Jupyter Lab with `jupyter lab`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [ ]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import ee

### Authenticate with Earth Engine

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, the next block will create a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then restart the notebook.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

### Elevation data example

Now let's make a simple example using [Shuttle Radar Topography Mission (SRTM)][srtm] elevation data. Here we create an `ee.Image` object referencing that dataset.

[srtm]: https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4

In [ ]:
image = ee.Image('CGIAR/SRTM90_V4')

Here `vis_params` consists of parameters that will be passed to the Earth Engine [`visParams` argument][visparams]. Any parameters that you could pass directly to Earth Engine in the code editor, you can also pass here to the `EarthEngineLayer`.

[visparams]: https://developers.google.com/earth-engine/image_visualization

In [ ]:
vis_params={
    "min": 0, 
    "max": 4000,
    "palette": ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}

Now we're ready to create the Pydeck layer. The `EarthEngineLayer` makes this simple. Just pass the Earth Engine object to the class.

Including the `id` argument isn't necessary when you only have one pydeck layer, but it is necessary to distinguish multiple layers, so it's good to get into the habit of including an `id` parameter.

In [ ]:
ee_layer = EarthEngineLayer(
    image,
    vis_params,
    id="SRTM_layer"
)

Then just pass this layer to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [ ]:
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)
r = pdk.Deck(
    layers=[ee_layer], 
    initial_view_state=view_state
)
r.show()

## Hillshade Example

As a slightly more in-depth example, let's use the SRTM dataset to calculate hillshading. This example comes from [giswqs/earthengine-py-notebooks][giswqs/earthengine-py-notebooks]:

[giswqs/earthengine-py-notebooks]: https://github.com/giswqs/earthengine-py-notebooks/blob/master/Visualization/hillshade.ipynb

In [ ]:
# Add Earth Engine dataset
import math

def Radians(img):
  return img.toFloat().multiply(math.pi).divide(180)

def Hillshade(az, ze, slope, aspect):
  """Compute hillshade for the given illumination az, el."""
  azimuth = Radians(ee.Image(az))
  zenith = Radians(ee.Image(ze))
  # Hillshade = cos(Azimuth - Aspect) * sin(Slope) * sin(Zenith) +
  #     cos(Zenith) * cos(Slope)
  return (azimuth.subtract(aspect).cos()
          .multiply(slope.sin())
          .multiply(zenith.sin())
          .add(
              zenith.cos().multiply(slope.cos())))

terrain = ee.Algorithms.Terrain(ee.Image('srtm90_v4'))
slope_img = Radians(terrain.select('slope'))
aspect_img = Radians(terrain.select('aspect'))

In [ ]:
ee_object = Hillshade(0, 60, slope_img, aspect_img)
ee_layer = EarthEngineLayer(ee_object)
view_state = pdk.ViewState(latitude=36.0756, longitude=-111.9987, zoom=7, bearing=0, pitch=30)
r = pdk.Deck(
    layers=[ee_layer], 
    initial_view_state=view_state
)
r.show()